In [1]:
using Flight.FlightCore.Systems
using Flight.FlightCore.Plotting

using Flight.FlightPhysics
using Flight.FlightComponents
using Flight.FlightAircraft.C172
using Flight.FlightAircraft.C172FBW
using Flight.FlightAircraft.C172FBW.FlightControl
using Flight.FlightComponents.Control.Discrete: PIDParams
using Flight.FlightComponents.Control.PIDOpt: Settings, Metrics, optimize_PID, build_PID, check_results

using UnPack
using ControlSystems
using RobustAndOptimalControl
using ComponentArrays
using LinearAlgebra

In [2]:
global_search = false

ac = Cessna172FBW(NED()) |> System #linearization requires NED kinematics

design_point = C172.TrimParameters()
# design_point = C172.TrimParameters(EAS = 28)
# design_point = C172.TrimParameters(EAS = 40)

#complete longitudinal model
lss_lon = Control.Continuous.LinearizedSS(ac, design_point; model = :lon);
P_lon = named_ss(lss_lon);

x_labels_lon = keys(lss_lon.x0) |> collect
y_labels_lon = keys(lss_lon.y0) |> collect
u_labels_lon = keys(lss_lon.u0) |> collect

lss_lon.A[:h, :] |> display
lss_lon.A[:, :h] |> display
lss_lon.C[:, :h] |> display


ComponentVector{Float64}(q = 0.0, θ = 52.57176296412945, v_x = 0.023550335280175867, v_z = -0.9997222499425569, h = 0.0, α_filt = 0.0, ω_eng = 0.0, thr_v = 0.0, thr_p = 0.0, ele_v = 0.0, ele_p = 0.0)

ComponentVector{Float64}(q = 6.60954228263296e-7, θ = -1.3093480728279614e-12, v_x = -1.9430531300316932e-5, v_z = 0.0009572946220013177, h = 0.0, α_filt = 0.0, ω_eng = 0.010967021085778062, thr_v = 0.0, thr_p = 0.0, ele_v = 0.0, ele_p = 0.0)

ComponentVector{Float64}(q = 0.0, θ = 0.0, v_x = 0.0, v_z = 0.0, h = 1.0, α_filt = 0.0, ω_eng = 0.0, thr_v = 0.0, thr_p = 0.0, ele_v = 0.0, ele_p = 0.0, f_x = -1.909930081594558e-5, f_z = 0.0009603649093991234, α = 0.0, EAS = -0.0024565174466087704, TAS = 0.0, γ = 0.0, climb_rate = 0.0, throttle_cmd = 0.0, elevator_cmd = 0.0)

As one would expect, $\dot{h}$ is nonzero for nonzero $\theta$. This will be a problem when designing a LQR tracker for which the choice of command variables involves non-zero steady state $\theta$. Since we cannot maintain constant altitude with an arbitrary $\theta$, true static equilibrium will not exist and the block $H_x F^{-1} G$ in the feedforward gain matrix required for tracking such command variables will be singular. This would not be an issue if we picked for instance $h$ and $EAS$ as command variables, because true equilibrium exists for an arbitrary combination of the two. However, this is not an appealing choice, because we would like to be able to control the pitch axis explicitly, rather than implicitly through $h$.

To handle this issue, we could partition the state and work with quasi-static equilibrium. However, since this is annoying, and the coupling of $h$ into the remaining states is very weak, we can simply drop $h$ from the state, design the desired LQR trackers on the reduced plant, and then connect them back to the full plant. This has a price to pay: ignoring the (small) effect of altitude causes (small) steady state errors when tracking climb-rate, EAS or throttle.

In [3]:
x_labels_red = copy(x_labels_lon)
y_labels_red = copy(y_labels_lon)
u_labels_red = copy(u_labels_lon)

x_labels_red = deleteat!(x_labels_red, findfirst(isequal(:h), x_labels_red))
y_labels_red = deleteat!(y_labels_red, findfirst(isequal(:h), y_labels_red))

#design model
lss_red = Control.Continuous.submodel(lss_lon; x = x_labels_red, u = u_labels_red, y = y_labels_red)
P_red = named_ss(lss_red);

Finally, we extract a model retaining only pitch dynamics, which will be used for pitch SAS design. Since we are only keeping elevator command as an input, we need to drop all engine-related states to keep the system controllable.

In [4]:
x_labels_pit = [:q, :θ, :v_x, :v_z, :α_filt, :ele_v, :ele_p]
y_labels_pit = vcat(x_labels_pit, [:f_x, :f_z, :α, :EAS, :TAS, :γ, :climb_rate, :elevator_cmd])
u_labels_pit = [:elevator_cmd,]

#pitch dynamics model
lss_pit = Control.Continuous.submodel(lss_lon; x = x_labels_pit, u = u_labels_pit, y = y_labels_pit)
P_pit = named_ss(lss_pit);

In [5]:
dampreport(P_lon)
dampreport(P_red)
dampreport(P_pit)
controllability(P_red)
controllability(P_pit)

|        Pole        |   Damping     |   Frequency   |   Frequency   | Time Constant |
|                    |    Ratio      |   (rad/sec)   |     (Hz)      |     (sec)     |
+--------------------+---------------+---------------+---------------+---------------+
| -0.000511          |  1            |  0.000511     |  8.14e-05     |  1.96e+03     |
| -0.0192 ±  0.233im |  0.0821       |  0.234        |  0.0372       |  52.1         |
| -5.54   ±   6.27im |  0.662        |  8.37         |  1.33         |  0.18         |
| -11.7              |  1            |  11.7         |  1.86         |  0.0857       |
| -18.8   ±   25.1im |  0.6          |  31.4         |  5            |  0.0531       |
| -18.8   ±   25.1im |  0.6          |  31.4         |  5            |  0.0531       |
| -47.2              |  1            |  47.2         |  7.51         |  0.0212       |
|        Pole        |   Damping     |   Frequency   |   Frequency   | Time Constant |
|                    |    Ratio      |   (r

(iscontrollable = true, ranks = [7, 7, 7, 7, 7, 7, 7], sigma_min = [0.004930613152567108, 3.213357064332774, 3.213357064332774, 0.4342631919592065, 0.4342631919592065, 0.03954932414967328, 0.03954932414967328])

In [6]:
cond(P_lon.A) |> display
cond(P_red.A) |> display
cond(P_pit.A) |> display

1.1793569208626257e8

707509.1616281078

190726.47973591764

## 1. Pitch Stability Augmentation

In [7]:
# ensure consistency in component selection and ordering between our reduced
# design model and FBWv1 avionics implementation for state and control vectors
@assert tuple(x_labels_red...) === propertynames(FlightControl.XLon())
@assert tuple(u_labels_red...) === propertynames(FlightControl.ULon())

x_trim = lss_red.x0
u_trim = lss_red.u0

n_x = length(x_labels_red)
n_u = length(u_labels_red)

UndefVarError: UndefVarError: `XLon` not defined

### 1.1. Decoupled

We first design a LQR from the pitch dynamics model. The resulting feedback loop will be connected to the reduced longitudinal model, padding the feedback matrix with zeros as required to fit the reduced model dimensions.

In [8]:
#compute a feedback matrix for the pitch dynamics model
@unpack v_x, v_z = lss_pit.x0
v_norm = norm([v_x, v_z])

#weight matrices
Q = ComponentVector(q = 1, θ = 20, v_x = 0.1/v_norm, v_z = 1/v_norm, α_filt = 0, ele_v = 0, ele_p = 0) |> diagm
R = ComponentVector(elevator_cmd = 2) |> diagm

#feedback gain matrix
C_fbk_pit = lqr(P_pit, Q, R)

#allocate a zero feedback matrix of the size required by the reduced model, and
#assign those components corresponding to the pitch dynamics feedback
#matrix
C_fbk_red = ComponentMatrix(zeros(n_u, n_x), Axis(u_labels_red), Axis(x_labels_red))
C_fbk_red[:elevator_cmd, x_labels_pit] .= vec(C_fbk_pit)

#connect the expanded feedback matrix to the reduced longitudinal model
u_labels_red_fbk = Symbol.(string.(u_labels_red) .* "_fbk")
u_labels_red_fwd = Symbol.(string.(u_labels_red) .* "_fwd")
u_labels_red_sum = Symbol.(string.(u_labels_red) .* "_sum")

C_fbk_red_ss = named_ss(ss(C_fbk_red), u = x_labels_red, y = u_labels_red_fbk)
throttle_sum = sumblock("throttle_cmd_sum = throttle_cmd_fwd - throttle_cmd_fbk")
elevator_sum = sumblock("elevator_cmd_sum = elevator_cmd_fwd - elevator_cmd_fbk")

connections = vcat(
    Pair.(x_labels_red, x_labels_red),
    Pair.(u_labels_red_fbk, u_labels_red_fbk),
    Pair.(u_labels_red_sum, u_labels_red),
    )

P_red_fbk = connect([P_red, throttle_sum, elevator_sum, C_fbk_red_ss],
    connections; w1 = u_labels_red_fwd, z1 = P_red.y)

dampreport(P_red_fbk)
display(C_fbk_red_ss)
# display(P_red_fbk)

UndefVarError: UndefVarError: `n_u` not defined

We now have a stabilized longitudinal model. We want this model to track direct throttle and elevator commands, so these are the command variables we shall consider. Because the feedback architecture is already embedded in the plant, we only need to determine the feedforward matrix from the open loop equilibrium. This is simply Stengel 6.2-13 (instead of the usual 6.2-35) without disturbances:
$$u_{sp} = B_{22} z_{sp} = C_{fwd} z_{sp}$$

In [9]:
z_labels = [:throttle_cmd, :elevator_cmd]
@assert tuple(z_labels...) === propertynames(FlightControl.ZLonThrEle())
z_trim = lss_red.y0[z_labels]
n_z = length(z_labels)
z_labels_sp = Symbol.(string.(z_labels) .* "_sp")

F = P_red_fbk.A
G = P_red_fbk.B
Hx = ComponentMatrix(P_red_fbk.C, Axis(P_red_fbk.y), Axis(P_red_fbk.x))[z_labels, :]
Hu = ComponentMatrix(P_red_fbk.D, Axis(P_red_fbk.y), Axis(P_red_fbk.x))[z_labels, :]

A = [F G; Hx Hu]
B = inv(A)
B_12 = B[1:n_x, n_x+1:end]
B_22 = B[n_x+1:end, n_x+1:end]

C_fwd_red = B_22
C_int_red = zeros(n_u, n_z) #no integral control

C_fwd_red_ss = named_ss(ss(C_fwd_red), u = z_labels_sp, y = u_labels_red_fwd)

connections = Pair.(u_labels_red_fwd, u_labels_red_fwd)

P_te = connect([P_red_fbk, C_fwd_red_ss],
    connections; w1 = z_labels_sp, z1 = P_red.y)

dampreport(P_te)
display(C_fwd_red)
# display(P_te)

UndefVarError: UndefVarError: `ZLonThrEle` not defined

In [10]:
e2e_SAS = P_te[:elevator_cmd, :elevator_cmd_sp]
e2θ_SAS = P_te[:θ, :elevator_cmd_sp]
e2t_SAS = P_te[:throttle_cmd, :elevator_cmd_sp]
e2v_SAS = P_te[:EAS, :elevator_cmd_sp]
e2θ_ol = P_red[:θ, :elevator_cmd]
step(e2e_SAS, 30) |> plot
step(e2θ_SAS, 30) |> plot!
# step(e2v_SAS, 30) |> plot!
step(e2θ_ol, 30) |> plot!
step(e2t_SAS, 30) |> plot!

UndefVarError: UndefVarError: `P_te` not defined

In [11]:
#we still see a slight coupling from throttle to elevator, but this is to be
#expected, because the engine can change the pitching moment and the pitch SAS
#tries to counteract this. however, the advantage of this design is that we
#retain explicit control over throttle_cmd, since it is not part of the feedback
#architecture
t2t_SAS = P_te[:throttle_cmd, :throttle_cmd_sp]
t2e_SAS = P_te[:elevator_cmd, :throttle_cmd_sp]
t2θ_SAS = P_te[:θ, :throttle_cmd_sp]
t2q_SAS = P_te[:q, :throttle_cmd_sp]
t2θ_ol = P_red[:θ, :throttle_cmd]
# e2t_SAS = P_red_te[:throttle_cmd, :elevator_cmd_sp]
# e2θ_ol = P_red[:θ, :elevator_cmd]
step(t2t_SAS, 30) |> plot
step(t2e_SAS, 30) |> plot!
step(t2θ_SAS, 30) |> plot!
step(t2q_SAS, 30) |> plot!
# step(t2θ_ol, 30) |> plot!
# step(e2t_SAS, 30) |> plot!
# step(e2θ_ol, 30) |> plot!

UndefVarError: UndefVarError: `P_te` not defined

### 1.2. Coupled

In [12]:
z_labels = [:throttle_cmd, :elevator_cmd]

@assert tuple(z_labels...) === propertynames(FlightControl.ZLonThrEle())
z_trim = lss_red.y0[z_labels]
n_z = length(z_labels)

F = lss_red.A
G = lss_red.B
Hx = lss_red.C[z_labels, :]
Hu = lss_red.D[z_labels, :]

@unpack v_x, v_z = lss_red.x0
v_norm = norm([v_x, v_z])

#weight matrices
Q = FlightControl.XLon(q = 1, θ = 30, v_x = 0.01/v_norm, v_z = 1/v_norm, α_filt = 0, ω_eng = 0, thr_v = 0.0, thr_p = 0, ele_v = 0, ele_p = 0) |> diagm
R = FlightControl.ULon(throttle_cmd = 2, elevator_cmd = 2) |> diagm

#feedback gain matrix
C_fbk = lqr(P_red, Q, R)

#forward gain matrix
A = [F G; Hx Hu]
B = inv(A)
B_12 = B[1:n_x, n_x+1:end]
B_22 = B[n_x+1:end, n_x+1:end]
C_fwd = B_22 + C_fbk * B_12

#no integral control
C_int = zeros(n_u, n_z)

#useful signal labels
u_labels_red_fbk = Symbol.(string.(u_labels_red) .* "_fbk")
u_labels_red_fwd = Symbol.(string.(u_labels_red) .* "_fwd")
u_labels_red_sum = Symbol.(string.(u_labels_red) .* "_sum")
z_labels_sp = Symbol.(string.(z_labels) .* "_sp")

C_fbk_ss = named_ss(ss(C_fbk), u = x_labels_red, y = u_labels_red_fbk)
C_fwd_ss = named_ss(ss(C_fwd), u = z_labels_sp, y = u_labels_red_fwd)

#summing junctions
throttle_sum = sumblock("throttle_cmd_sum = throttle_cmd_fwd- throttle_cmd_fbk")
elevator_sum = sumblock("elevator_cmd_sum = elevator_cmd_fwd - elevator_cmd_fbk")

connections = vcat(
    Pair.(x_labels_red, x_labels_red),
    Pair.(u_labels_red_fbk, u_labels_red_fbk),
    Pair.(u_labels_red_sum, u_labels_red),
    Pair.(u_labels_red_fwd, u_labels_red_fwd)
    )

#and now connect it back to the complete longitudinal dynamics
# P_te_coupled_lon = connect([P_lon, elevator_sum, throttle_sum, C_fbk_ss, C_fwd_ss],
#     connections; w1 = z_labels_sp, z1 = P_lon.y)

P_te_coupled = connect([P_red, elevator_sum, throttle_sum, C_fbk_ss, C_fwd_ss],
    connections; w1 = z_labels_sp, z1 = P_red.y)

dampreport(P_te_coupled)
# dampreport(P_te2)
display(C_fbk_ss)
display(C_fwd_ss)

UndefVarError: UndefVarError: `ZLonThrEle` not defined

In [13]:
e2e_SAS = P_te_coupled[:elevator_cmd, :elevator_cmd_sp]
e2θ_SAS = P_te_coupled[:θ, :elevator_cmd_sp]
e2t_SAS = P_te_coupled[:throttle_cmd, :elevator_cmd_sp]
e2θ_ol = P_red[:θ, :elevator_cmd]
step(e2e_SAS, 20) |> plot
step(e2θ_SAS, 20) |> plot!
step(e2t_SAS, 20) |> plot!
step(e2θ_ol, 20) |> plot!

UndefVarError: UndefVarError: `P_te_coupled` not defined

In [14]:
t2t_SAS = P_te_coupled[:throttle_cmd, :throttle_cmd_sp]
t2v_SAS = P_te_coupled[:EAS, :throttle_cmd_sp]
t2θ_SAS = P_te_coupled[:θ, :throttle_cmd_sp]
t2q_SAS = P_te_coupled[:q, :throttle_cmd_sp]
step(t2t_SAS, 30) |> plot
# step(t2v_SAS, 30) |> plot!
step(t2θ_SAS, 30) |> plot!
step(t2q_SAS, 30) |> plot!

UndefVarError: UndefVarError: `P_te_coupled` not defined

## 2. $q \rightarrow e$

We now design a pitch rate to elevator CAS on top of the stabilized plant using classical SISO techniques.

In [15]:
P_e2q = P_te[:q, :elevator_cmd_sp]
marginplot(P_e2q)

UndefVarError: UndefVarError: `P_te` not defined

In [16]:
nyquistplot(P_e2q; unit_circle = true, Ms_circles = [1.0])
plot!(ylims = (-2,2), xlims = (-2,2))

UndefVarError: UndefVarError: `P_e2q` not defined

In [17]:
q2e_int = tf(1, [1, 0]) |> ss
P_q2e_opt = series(q2e_int, ss(P_e2q))

t_sim_q2e = 10
lower_bounds = PIDParams(; k_p = 0.1, k_i = 0.0, k_d = 0.0, τ_f = 0.01)
upper_bounds = PIDParams(; k_p = 10.0, k_i = 35.0, k_d = 1.5, τ_f = 0.01)
settings = Settings(; t_sim = t_sim_q2e, lower_bounds, upper_bounds)
weights = Metrics(; Ms = 1, ∫e = 10, ef = 2, ∫u = 0.1, up = 0.00)
params_0 = PIDParams(; k_p = 2.0, k_i = 15, k_d = 0.4, τ_f = 0.01)

q2e_results = optimize_PID(P_q2e_opt; params_0, settings, weights, global_search)

@unpack k_p, k_i, k_d, T_i, T_d = q2e_results.params
@unpack k_p, k_i, k_d, T_i, T_d = q2e_results.params
@show k_p, k_i, k_d, T_i, T_d
@show q2e_results.metrics
@show q2e_results.exit_flag

UndefVarError: UndefVarError: `P_e2q` not defined

In [18]:
q2e_PID = build_PID(q2e_results.params)
# q2e_PID = build_PID(PIDParams(; k_p = 1.594, k_i = 15.216, k_d = 0.102))
C_q2e = named_ss(series(q2e_int, q2e_PID), :C_q2e; u = :q_err, y = :elevator_cmd_sp);

UndefVarError: UndefVarError: `q2e_results` not defined

In [19]:
#steady-state error for a unit step input, now zero
L_q2e = series(C_q2e, P_e2q)
ε_q2e_cl = 1/(1+tf(L_q2e))
@show ε_q2e_cl(0)

T_q2e = output_comp_sensitivity(P_e2q, C_q2e) #this is the closed loop transfer function
T_q2e_step = step(T_q2e, t_sim_q2e)
stepinfo(T_q2e_step) |> display
T_q2e_step |> plot

UndefVarError: UndefVarError: `C_q2e` not defined

In [20]:
CS_q2e = G_CS(P_e2q, C_q2e)
CS_q2e_step = step(CS_q2e, t_sim_q2e)
CS_q2e_step |> plot

UndefVarError: UndefVarError: `P_e2q` not defined

In [21]:
marginplot(L_q2e)

UndefVarError: UndefVarError: `L_q2e` not defined

In [22]:
nyquistplot(L_q2e; unit_circle = true, Ms_circles = [1.0])
plot!(ylims = (-2,2), xlims = (-2,2))

UndefVarError: UndefVarError: `L_q2e` not defined

In [23]:
gangoffourplot(P_e2q, C_q2e)

UndefVarError: UndefVarError: `P_e2q` not defined

In [24]:
q2e_sum = sumblock("q_err = q_sp - q")
P_tq = connect([P_te, q2e_sum, C_q2e], [:q_err=>:q_err, :q=>:q, :elevator_cmd_sp=>:elevator_cmd_sp], w1 = [:throttle_cmd_sp, :q_sp], z1 = P_te.y)

#verify we get the same response as with the SISO closed loop
qsp2q = P_tq[:q, :q_sp]
qsp2t = P_tq[:throttle_cmd, :q_sp]
qsp2e = P_tq[:elevator_cmd, :q_sp]
step(qsp2q, t_sim_q2e) |> plot
step(qsp2t, t_sim_q2e) |> plot!
# step(qsp2e, t_sim_q2e) |> plot!
#and we see that the response matches that of the SISO design

UndefVarError: UndefVarError: `P_te` not defined

In [25]:
dampreport(P_tq)

UndefVarError: UndefVarError: `P_tq` not defined

To avoid transients on mode changes, we want the initial output of the pitch rate loop to match the current value of the elevator command setpoint at the pitch SAS input, $e_{cmd,sp}$.

Whenever pitch rate mode is enabled after a mode change, we first reset both the integrator and the PID. Then we set PID integrator's state to the appropriate value to achieve the required $e_{cmd,sp}$. We have:
$$ y_{int} = 0 $$
$$ y_{pid} = k_p y_{int} + k_i x_{i0} = k_i x_{i0} = e_{cmd,sp} $$

Therefore:
$$x_{i0} = e_{cmd, sp}/k_i$$

## 3. $\theta \rightarrow q$


We start with the pitch-rate augmented plant, with throttle and elevator inputs.

In [26]:
P_q2θ = P_tq[:θ, :q_sp]
step(P_q2θ, 10) |> plot

UndefVarError: UndefVarError: `P_tq` not defined

In [27]:
marginplot(P_q2θ)

UndefVarError: UndefVarError: `P_q2θ` not defined

In [28]:
nyquistplot(P_q2θ; unit_circle = true, Ms_circles = [1.0])
plot!(ylims = (-2,2), xlims = (-2,2))

UndefVarError: UndefVarError: `P_q2θ` not defined

In [29]:
k_p_θ2q = 1
T_i_θ2q = Inf
T_d_θ2q = 0.0
τ_f_θ2q = 0.01
@show k_i_θ2q= k_p_θ2q/ T_i_θ2q
@show k_d_θ2q= k_p_θ2q* T_d_θ2q

θ2q_pid = k_p_θ2q * (1 + 1/T_i_θ2q * tf(1, [1, 0]) + T_d_θ2q * tf([1, 0], [τ_f_θ2q, 1]))
C_θ2q = θ2q_pid
C_θ2q = named_ss(ss(C_θ2q), :C_θ2q; u = :θ_err, y = :q_sp);
L_θ2q = series(C_θ2q, P_q2θ);

k_i_θ2q = k_p_θ2q / T_i_θ2q = 0.0
k_d_θ2q = k_p_θ2q * T_d_θ2q = 0.0


UndefVarError: UndefVarError: `P_q2θ` not defined

Testing this simple unit gain feedback design, we find out that it works well for the whole EAS range, so for now it does not seem warranted to use a gain scheduled PID compensator.

In [30]:
t_sim_θ2q = 10

10

In [31]:
nyquistplot(L_θ2q; unit_circle = true, Ms_circles = [1.0])
plot!(ylims = (-2,2), xlims = (-2,2))

UndefVarError: UndefVarError: `L_θ2q` not defined

In [32]:
marginplot(L_θ2q)

UndefVarError: UndefVarError: `L_θ2q` not defined

In [33]:
T_θ2q = output_comp_sensitivity(P_q2θ, C_θ2q)
T_θ2q_step = step(T_θ2q, t_sim_θ2q)
stepinfo(T_θ2q_step) |> display
T_θ2q_step |> plot

UndefVarError: UndefVarError: `P_q2θ` not defined

In [34]:
gangoffourplot(P_q2θ, C_θ2q)

UndefVarError: UndefVarError: `P_q2θ` not defined

In [35]:
θ2q_sum = sumblock("θ_err = θ_sp - θ")
P_tθ = connect([P_tq, θ2q_sum, C_θ2q], [:θ_err=>:θ_err, :θ=>:θ, :q_sp=>:q_sp], w1 = [:throttle_cmd_sp, :θ_sp], z1 = P_tq.y);

#check we get the same response as with the SISO closed loop
θsp2θ = P_tθ[:θ, :θ_sp]
θsp2t = P_tθ[:throttle_cmd, :θ_sp]
θsp2e = P_tθ[:elevator_cmd, :θ_sp]
step(θsp2θ, t_sim_θ2q) |> plot
step(θsp2t, t_sim_θ2q) |> plot!
# step(θsp2e, t_sim_θ2q) |> plot!

UndefVarError: UndefVarError: `P_tq` not defined

Our linear model comes from a horizontal, wings level flight condition, in which:
$$\dot{\theta} = q$$
However, in the general case we have the following kinematic relations:
$$
q = \dot{\theta} \cos \phi + \dot{\psi} \sin \phi \cos \theta \\
\dot{\psi} = \dfrac{1}{\cos \theta} (q \sin \phi + r \cos \phi)
$$

Substituting the second equation into the first and solving for q we arrive at:
$$
q(\dot{\theta}, r, \phi) = \dfrac{1}{\cos \phi}(\dot{\theta} + r \tan \phi)
$$

This shows that, when we have nonzero $r$ and nonzero $\phi$ (for example, during a turn), maintaining $\dot{\theta} = 0$ requires nonzero $q$. Also, as $\phi$ increases, the gain from $\dot{\theta}$ to $q$ decreases.

Since the $\theta$ controller design assumes $q = \dot{\theta}$, the $q_{sp}$ output by the $\theta$ controller should actually be interpreted as $\dot{\theta}_{sp}$ (or equivalently, the $q_{sp}$ we would need to track the desired $\theta$ in the absence of $r$ and $\phi$). The actual $q_{sp}$ required is given by:
$$q_{sp}=q(\dot{\theta}_{sp}, r, \phi)$$

What would happen during a turn if we didn't introduce this feedforward correction? Since the $\theta$ controller is purely proportional, it can only generate a nonzero output for nonzero $\theta$ error. Therefore, the necessary steady state $q = r \tan \phi$ required by the turn can only be achieved by the $\theta$ controller through a nonzero steady state $\theta$ error. This means that turning would always produce a nonzero steady-state $\theta$ error. The higher the turn rate, the larger the error.

To avoid transients on mode changes, we want the initial $\theta$ loop to produce $\dot{\theta}_{sp} = 0$ when $\theta_{sp}$ is initialized to the current $\theta$. Since this is only a proportional compensator this is unconditionally true:
$$\dot{\theta}_{sp} = k_p (\theta_{sp} - \theta)$$

## 4. EAS $\rightarrow \theta$


We start from the $\theta$-augmented plant. We leave the throttle alone and design a EAS to $\theta$ SISO loop.

In [36]:
P_θ2v = P_tθ[:EAS, :θ_sp]
step(P_θ2v, 60) |> plot

UndefVarError: UndefVarError: `P_tθ` not defined

As expected, we have negative DC gain, so the compensator will require a sign inversion. For design, we can move this inversion to the plant.

In [37]:
zpk(P_θ2v) |> display
P_θ2v_opt = -P_θ2v;

UndefVarError: UndefVarError: `P_θ2v` not defined

In [38]:
nyquistplot(P_θ2v_opt; unit_circle = true, Ms_circles = [1.0])
plot!(ylims = (-2,2), xlims = (-2,2))

UndefVarError: UndefVarError: `P_θ2v_opt` not defined

In [39]:
# k_p_v2θ = .04
# T_i_v2θ = 5
# T_d_v2θ = 0.0
# τ_f_v2θ = 0.01
# @show k_i_v2θ= k_p_v2θ/ T_i_v2θ
# @show k_d_v2θ= k_p_v2θ* T_d_v2θ
# v2θ_pid = k_p_v2θ * (1 + 1/T_i_v2θ * tf(1, [1, 0]) + T_d_v2θ * tf([1, 0], [τ_f_v2θ, 1]))
# C_v2θ = -v2θ_pid

t_sim_v2θ = 20
lower_bounds = PIDParams(; k_p = 0.01, k_i = 0.000, k_d = 0.0, τ_f = 0.01)
upper_bounds = PIDParams(; k_p = 0.2, k_i = 0.05, k_d = 0.0, τ_f = 0.01)
settings = Settings(; t_sim = t_sim_v2θ, lower_bounds, upper_bounds)
weights = Metrics(; Ms = 2.0, ∫e = 5.0, ef = 1.0, ∫u = 0.0, up = 0.0)
params_0 = PIDParams(; k_p = 0.05, k_i = 0.01, k_d = 0.0, τ_f = 0.01)

v2θ_results = optimize_PID(P_θ2v_opt; params_0, settings, weights, global_search)
v2θ_pid = build_PID(v2θ_results.params)
C_v2θ = -v2θ_pid
C_v2θ = named_ss(ss(C_v2θ), :C_v2θ; u = :EAS_err, y = :θ_sp)
L_v2θ = series(C_v2θ, P_θ2v);

@unpack k_p, k_i, k_d, τ_f, T_i, T_d = v2θ_results.params
@show k_p, k_i, k_d, τ_f, T_i, T_d
@show v2θ_results.metrics
@show v2θ_results.exit_flag;

UndefVarError: UndefVarError: `P_θ2v_opt` not defined

In [40]:
nyquistplot(L_v2θ; unit_circle = true, Ms_circles = [1.0])
plot!(ylims = (-2,2), xlims = (-2,2))


UndefVarError: UndefVarError: `L_v2θ` not defined

In [41]:
marginplot(L_v2θ)

UndefVarError: UndefVarError: `L_v2θ` not defined

In [42]:
T_v2θ = output_comp_sensitivity(P_θ2v, C_v2θ)
T_v2θ_step = step(T_v2θ, t_sim_v2θ)
stepinfo(T_v2θ_step) |> display
T_v2θ_step |> plot

UndefVarError: UndefVarError: `P_θ2v` not defined

In [43]:
gangoffourplot(P_θ2v, C_v2θ)

UndefVarError: UndefVarError: `P_θ2v` not defined

In [44]:
v2θ_sum = sumblock("EAS_err = EAS_sp - EAS")
P_tv = connect([P_tθ, v2θ_sum, C_v2θ], [:EAS_err=>:EAS_err, :EAS=>:EAS, :θ_sp=>:θ_sp], w1 = [:throttle_cmd_sp, :EAS_sp], z1 = P_tθ.y)

#check we get the same response as with the SISO closed loop
EASsp2EAS = P_tv[:EAS, :EAS_sp]
EASsp2e = P_tv[:elevator_cmd, :EAS_sp]
EASsp2θ = P_tv[:θ, :EAS_sp]
step(EASsp2EAS, t_sim_v2θ) |> plot
step(EASsp2e, t_sim_v2θ) |> plot!
step(EASsp2θ, t_sim_v2θ) |> plot!

UndefVarError: UndefVarError: `P_tθ` not defined

To avoid transients on mode changes, we want the initial $\theta_{sp}$ output from the $v2\theta$ controller to match the current value of pitch angle $\theta$ when $EAS_{sp}$ matches the current $EAS$.

Whenever $v2\theta$ is enabled after a mode change, we reset the PID. Then we set the PID integrator's state to the appropriate value to achieve the required $\theta$. Noting the sign inversion, we have:
$$ y_{pid} = k_p (EAS_{sp} - EAS) + k_i x_{i0} = k_i x_{i0} $$
$$ \theta_{sp} = -y_{pid}$$

Therefore:
$$x_{i0} = y_{pid} / k_i = -\theta_{sp}/k_i = -\theta/k_i$$

## 5. $EAS \rightarrow t$

To design a SISO EAS-to-throttle controller, we should start from a plant with pitch control augmentation. The throttle-to-EAS response of the raw plant is unsuitable.

In [45]:
P_t2v = P_tq[:EAS, :throttle_cmd]
dampreport(P_t2v)
step(P_t2v, 10) |> plot

UndefVarError: UndefVarError: `P_tq` not defined

In [46]:
#error transfer function for a unit feedback closed loop and a unit step input
marginplot(P_t2v)

UndefVarError: UndefVarError: `P_t2v` not defined

In [47]:
nyquistplot(P_e2q; unit_circle = true, Ms_circles = [1.0])
plot!(ylims = (-2,2), xlims = (-2,2))

UndefVarError: UndefVarError: `P_e2q` not defined

In [48]:
t_sim_v2t = 10
lower_bounds = PIDParams(; k_p = 0.1, k_i = 0.0, k_d = 0.0, τ_f = 0.01)
upper_bounds = PIDParams(; k_p = 1.0, k_i = 0.5, k_d = 0.0, τ_f = 0.01)
settings = Settings(; t_sim = t_sim_v2t, maxeval = 5000, lower_bounds, upper_bounds)
weights = Metrics(; Ms = 2.0, ∫e = 5.0, ef = 1.0, ∫u = 0.0, up = 0.0)
params_0 = PIDParams(; k_p = 0.2, k_i = 0.1, k_d = 0.0, τ_f = 0.01)

v2t_results = optimize_PID(P_t2v; params_0, settings, weights, global_search)
v2t_PID = build_PID(v2t_results.params)

C_v2t = named_ss(ss(v2t_PID), :C_v2t; u = :EAS_err, y = :throttle_cmd_sp)
L_v2t = series(C_v2t, P_t2v)

@unpack k_p, k_i, k_d, T_i, T_d = v2t_results.params
@show k_p, k_i, k_d, T_i, T_d
@show v2t_results.metrics
@show v2t_results.exit_flag

UndefVarError: UndefVarError: `P_t2v` not defined

In [49]:
marginplot(L_v2t)

UndefVarError: UndefVarError: `L_v2t` not defined

In [50]:
nyquistplot(L_v2t; unit_circle = true, Ms_circles = [1.0])
plot!(ylims = (-2,2), xlims = (-2,2))

UndefVarError: UndefVarError: `L_v2t` not defined

In [51]:
T_v2t = output_comp_sensitivity(P_t2v, C_v2t)
T_v2t_step = step(T_v2t, t_sim_v2t)
stepinfo(T_v2t_step) |> display
T_v2t_step |> plot

UndefVarError: UndefVarError: `P_t2v` not defined

In [52]:
CS_v2t = G_CS(P_t2v, C_v2t)
CS_v2t_step = step(CS_v2t, t_sim_v2t)
CS_v2t_step |> plot!

UndefVarError: UndefVarError: `P_t2v` not defined

In [53]:
gangoffourplot(P_t2v, C_v2t)

UndefVarError: UndefVarError: `P_t2v` not defined

In [54]:
v2t_sum = sumblock("EAS_err = EAS_sp - EAS")
P_vq = connect([P_tq, v2t_sum, C_v2t], [:EAS_err=>:EAS_err, :EAS=>:EAS, :throttle_cmd_sp=>:throttle_cmd_sp], w1 = [:EAS_sp, :q_sp], z1 = P_tq.y)

#check we get the same response as with the SISO closed loop
EASsp2EAS = P_vq[:EAS, :EAS_sp]
EASsp2thr = P_vq[:throttle_cmd, :EAS_sp]
EASsp2θ = P_vq[:θ, :EAS_sp]
step(EASsp2EAS, t_sim_v2t) |> plot
step(EASsp2thr, t_sim_v2t) |> plot!
step(EASsp2θ, t_sim_v2t) |> plot!

UndefVarError: UndefVarError: `P_tq` not defined

To avoid transients on mode changes, we want the initial throttle command setpoint $thr_{sp}$ output from the $v2t$ controller to match the current value of throttle command $thr$ when $EAS_{sp}$ matches the current $EAS$. Whenever $v2t$ is enabled after a mode change, we reset the PID. Then we set the PID integrator's state to the appropriate value to achieve the required $thr$. We have:
$$thr_{sp} = y_{pid} = k_p (EAS_{sp} - EAS) + k_i x_{i0} = k_i x_{i0} = thr $$

Therefore:
$$x_{i0} = thr/k_i$$

## 6. EAS + Climb Rate Control

We start from scratch from the non-stabilized reduced longitudinal dynamics. This will be a true MIMO compensator mode with integral control in both command variables.

In [55]:
#define command variables
z_labels = [:EAS, :climb_rate]
@assert tuple(z_labels...) === propertynames(FlightControl.ZLonEASClm())
z_trim = lss_red.y0[z_labels]
n_z = length(z_labels)

z_labels_sp = Symbol.(string.(z_labels) .* "_sp")


UndefVarError: UndefVarError: `ZLonEASClm` not defined

In [56]:
F = lss_red.A
G = lss_red.B
Hx = lss_red.C[z_labels, :]
Hu = lss_red.D[z_labels, :]
display(Hx)
display(Hu)

#define the blocks corresponding to the subset of the command variables for
#which integral compensation is required. in this case, both
Hx_int = Hx[z_labels, :]
Hu_int = Hu[z_labels, :]
n_int, _ = size(Hx_int)

F_aug = [F zeros(n_x, n_int); Hx_int zeros(n_int, n_int)]
G_aug = [G; Hu_int]
Hx_aug = [Hx zeros(n_z, n_int)]
Hu_aug = Hu

P_aug = ss(F_aug, G_aug, Hx_aug, Hu_aug)
controllability(F_aug, G_aug)

2×10 ComponentMatrix{Float64} with axes Axis(EAS = 1, climb_rate = 2) × Axis(q = 1, θ = 2, v_x = 3, v_z = 4, α_filt = 5, ω_eng = 6, thr_v = 7, thr_p = 8, ele_v = 9, ele_p = 10)
 0.0   0.0     0.950817    0.0223979  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  52.5718  0.0235503  -0.999722   0.0  0.0  0.0  0.0  0.0  0.0

2×2 ComponentMatrix{Float64} with axes Axis(EAS = 1, climb_rate = 2) × Axis(throttle_cmd = 1, elevator_cmd = 2)
 0.0  0.0
 0.0  0.0

UndefVarError: UndefVarError: `n_x` not defined

In [57]:
@unpack v_x, v_z = lss_red.x0
v_norm = norm([v_x, v_z])

#weight matrices
Q = ComponentVector(q = 1, θ = 100, v_x = 10/v_norm, v_z = 1/v_norm, α_filt = 1, ω_eng = 0, thr_v = 0.0, thr_p = 0, ele_v = 0, ele_p = 0, ξ_EAS = 0.005, ξ_climb_rate = 0.001) |> diagm
R = FlightControl.ULon(throttle_cmd = 1, elevator_cmd = 1) |> diagm

#compute gain matrix
C_aug = lqr(P_aug, Q, R)

# quickly construct the closed-loop augmented system to check the resulting
# eigenvalues, we'll do the connections properly later
F_aug_cl = F_aug - G_aug * C_aug
P_aug_cl = ss(F_aug_cl, G_aug, Hx_aug, Hu_aug)

dampreport(P_aug)
dampreport(P_aug_cl)

UndefVarError: UndefVarError: `ULon` not defined

In [58]:
#extract system state and integrator blocks from the feedback matrix
C_x = C_aug[:, 1:n_x]
C_ξ = C_aug[:, n_x+1:end]
display(C_ξ)

UndefVarError: UndefVarError: `C_aug` not defined

In [59]:
A = [F G; Hx Hu]
B = inv(A)
B_12 = B[1:n_x, n_x+1:end]
B_22 = B[n_x+1:end, n_x+1:end]

C_fbk = C_x
C_fwd = B_22 + C_x * B_12
C_int = C_ξ

#some useful signal labels
u_labels_red_fbk = Symbol.(string.(u_labels_red) .* "_fbk")
u_labels_red_fwd = Symbol.(string.(u_labels_red) .* "_fwd")
u_labels_red_sum = Symbol.(string.(u_labels_red) .* "_sum")
u_labels_red_int_u = Symbol.(string.(u_labels_red) .* "_int_u")
u_labels_red_int = Symbol.(string.(u_labels_red) .* "_int")
u_labels_red_ξ = Symbol.(string.(u_labels_red) .* "_ξ")

z_labels_sp = Symbol.(string.(z_labels) .* "_sp")
z_labels_sp1 = Symbol.(string.(z_labels) .* "_sp1")
z_labels_sp2 = Symbol.(string.(z_labels) .* "_sp2")
z_labels_err = Symbol.(string.(z_labels) .* "_err")
z_labels_sum = Symbol.(string.(z_labels) .* "_sum")
z_labels_sp_fwd = Symbol.(string.(z_labels) .* "_sp_fwd")
z_labels_sp_sum = Symbol.(string.(z_labels) .* "_sp_sum")

C_fbk_ss = named_ss(ss(C_fbk), u = x_labels_red, y = u_labels_red_fbk)
C_fwd_ss = named_ss(ss(C_fwd), u = z_labels_sp_fwd, y = u_labels_red_fwd)
C_int_ss = named_ss(ss(C_int), u = z_labels_err, y = u_labels_red_int_u)

int_ss = named_ss(ss(tf(1, [1,0])) .* I(2),
                    x = u_labels_red_ξ,
                    u = u_labels_red_int_u,
                    y = u_labels_red_int);

EAS_err_sum = sumblock("EAS_err = EAS_sum - EAS_sp_sum")
climb_rate_err_sum = sumblock("climb_rate_err = climb_rate_sum - climb_rate_sp_sum")

throttle_cmd_sum = sumblock("throttle_cmd_sum = throttle_cmd_fwd - throttle_cmd_fbk - throttle_cmd_int")
elevator_cmd_sum = sumblock("elevator_cmd_sum = elevator_cmd_fwd - elevator_cmd_fbk - elevator_cmd_int")

EAS_sp_splitter = splitter(:EAS_sp, 2)
climb_rate_sp_splitter = splitter(:climb_rate_sp, 2)

connections = vcat(
    Pair.(x_labels_red, x_labels_red),
    Pair.(z_labels, z_labels_sum),
    Pair.(z_labels_sp1, z_labels_sp_sum),
    Pair.(z_labels_sp2, z_labels_sp_fwd),
    Pair.(z_labels_err, z_labels_err),
    Pair.(u_labels_red_sum, u_labels_red),
    Pair.(u_labels_red_fwd, u_labels_red_fwd),
    Pair.(u_labels_red_fbk, u_labels_red_fbk),
    Pair.(u_labels_red_int, u_labels_red_int),
    Pair.(u_labels_red_int_u, u_labels_red_int_u),
    )

P_vc = connect([P_lon, int_ss, C_fwd_ss, C_fbk_ss, C_int_ss,
                    EAS_err_sum, climb_rate_err_sum,
                    throttle_cmd_sum, elevator_cmd_sum,
                    EAS_sp_splitter, climb_rate_sp_splitter], connections;
                    w1 = z_labels_sp, z1 = P_lon.y)

# P_vc = connect([P_red, int_ss, C_fwd_ss, C_fbk_ss, C_int_ss,
#                     EAS_err_sum, climb_rate_err_sum,
#                     throttle_cmd_sum, elevator_cmd_sum,
#                     EAS_sp_splitter, climb_rate_sp_splitter], connections;
#                     w1 = z_labels_sp, z1 = P_red.y)
dampreport(P_vc)
display(C_fbk_ss)
display(C_fwd_ss)
display(C_int_ss)
display(int_ss)

UndefVarError: UndefVarError: `n_x` not defined

In [60]:
EASsp2EAS = P_vc[:EAS, :EAS_sp]
EASsp2thr = P_vc[:throttle_cmd, :EAS_sp]
EASsp2ele = P_vc[:elevator_cmd, :EAS_sp]
EASsp2q = P_vc[:q, :EAS_sp]
step(EASsp2EAS, 30) |> plot
step(EASsp2thr, 30) |> plot!
step(EASsp2ele, 30) |> plot!
step(EASsp2q, 30) |> plot!

UndefVarError: UndefVarError: `P_vc` not defined

In [61]:
crsp2cr = P_vc[:climb_rate, :climb_rate_sp]
crsp2EAS = P_vc[:EAS, :climb_rate_sp]
# crsp2ele = P_vc[:elevator_cmd, :climb_rate_sp]
crsp2thr = P_vc[:throttle_cmd, :climb_rate_sp]
step(crsp2cr, 30) |> plot
step(crsp2EAS, 30) |> plot!
step(crsp2thr, 30) |> plot!
# step(crsp2ele, 30) |> plot!
# step(crsp2thr, 30) |> plot!

UndefVarError: UndefVarError: `P_vc` not defined

In [62]:
marginplot(EASsp2EAS)

UndefVarError: UndefVarError: `EASsp2EAS` not defined

In [63]:
marginplot(crsp2cr)

UndefVarError: UndefVarError: `crsp2cr` not defined

## 7. $h \rightarrow c$

We start from scratch from the EAS + climb rate plant.

In [64]:
P_c2h = P_vc[:h, :climb_rate_sp]
step(P_c2h, 10) |> plot

UndefVarError: UndefVarError: `P_vc` not defined

In [65]:
marginplot(P_c2h)

UndefVarError: UndefVarError: `P_c2h` not defined

In [66]:
nyquistplot(P_c2h; unit_circle = true, Ms_circles = [1.0])
plot!(ylims = (-2,2), xlims = (-2,2))

UndefVarError: UndefVarError: `P_c2h` not defined

In [67]:
k_h2c = 0.2
C_h2c = named_ss(ss(k_h2c), :C_h2c; u = :h_err, y = :climb_rate_sp);
L_h2c = series(C_h2c, P_c2h);

UndefVarError: UndefVarError: `P_c2h` not defined

In [68]:
marginplot(L_h2c)

UndefVarError: UndefVarError: `L_h2c` not defined

In [69]:
nyquistplot(L_h2c; unit_circle = true, Ms_circles = [1.0])
plot!(ylims = (-2,2), xlims = (-2,2))

UndefVarError: UndefVarError: `L_h2c` not defined

In [70]:
gangoffourplot(P_c2h, C_h2c)

UndefVarError: UndefVarError: `P_c2h` not defined